This tutorial will show you how to [install Milvus from source code](https://github.com/milvus-io/milvus/blob/master/INSTALL.md) and run the [python code](https://milvus-io.github.io/milvus-sdk-python/pythondoc/v0.2.14/index.html).

## Install Milvus from source code

### Install Camke

- Get CMake 3.12 or higher. Here we get CMake 3.12.

In [ ]:
! wget https://cmake.org/files/v3.12/cmake-3.12.2-Linux-x86_64.tar.gz

--2020-09-18 01:54:27--  https://cmake.org/files/v3.12/cmake-3.12.2-Linux-x86_64.tar.gz
Resolving cmake.org (cmake.org)... 66.194.253.19
Connecting to cmake.org (cmake.org)|66.194.253.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34950376 (33M) [application/x-gzip]
Saving to: ‘cmake-3.12.2-Linux-x86_64.tar.gz’

cmake-3.12.2-Linux- 100%[===================>]  33.33M  18.9MB/s    in 1.8s    

2020-09-18 01:54:30 (18.9 MB/s) - ‘cmake-3.12.2-Linux-x86_64.tar.gz’ saved [34950376/34950376]



- Extract the file and install CMake.

In [ ]:
! tar zxvf cmake-3.12.2-Linux-x86_64.tar.gz
! mv cmake-3.12.2-Linux-x86_64 /opt/cmake-3.12.2
! ln -sf /opt/cmake-3.12.2/bin/* /usr/bin/

流式输出内容被截断，只能显示最后 5000 行内容。
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_COMPILE_PDB_OUTPUT_DIRECTORY.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_LANG_ARCHIVE_APPEND.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_SHARED_LINKER_FLAGS_INIT.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_LANG_COMPILE_OBJECT.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_EXE_LINKER_FLAGS.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_NINJA_OUTPUT_PATH_PREFIX.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/MSVC10.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_BUILD_WITH_INSTALL_RPATH.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_COMPILER_IS_GNUCC.rst.txt
cmake-3.12.2-Linux-x86_64/doc/cmake/html/_sources/variable/CMAKE_SUBLIME_TEXT_2_EXCLUDE_BUILD_TREE.rst.txt
cmake-3.12.2-Linux-x86_64/doc

- Show the version.

In [ ]:
!cmake --version

cmake version 3.12.0

CMake suite maintained and supported by Kitware (kitware.com/cmake).


### Download the code and Compile

Step 1 Download Milvus source code

In [ ]:
!git clone -b 0.10.2 https://github.com/milvus-io/milvus.git

Cloning into 'milvus'...
remote: Enumerating objects: 63457, done.
remote: Total 63457 (delta 0), reused 0 (delta 0), pack-reused 63457
Receiving objects: 100% (63457/63457), 26.38 MiB | 13.84 MiB/s, done.
Resolving deltas: 100% (46884/46884), done.


Step 2 Install dependencies

In [ ]:
% cd /content/milvus/core
! ./ubuntu_build_deps.sh

/content/milvus/core
--2020-09-18 01:54:40--  https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB
Resolving apt.repos.intel.com (apt.repos.intel.com)... 23.222.59.186, 2a02:26f0:fe00:1ae::4b23, 2a02:26f0:fe00:1bb::4b23
Connecting to apt.repos.intel.com (apt.repos.intel.com)|23.222.59.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1914 (1.9K) [binary/octet-stream]
Saving to: ‘/tmp/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB’

GPG-PUB-KEY-INTEL-S 100%[===================>]   1.87K  --.-KB/s    in 0s      

2020-09-18 01:54:40 (257 MB/s) - ‘/tmp/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB’ saved [1914/1914]

OK
Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 https://cloud.r-project.org/bin/linux/

Step 3 Build Milvus source code

In [ ]:
% cd /content/milvus/core  
!ls
!./build.sh -t Release
# If you want to build GPU version, add -g option, and Edit the Notebook settings with GPU
# !./build.sh -t Release -g

流式输出内容被截断，只能显示最后 5000 行内容。
[C]       Compiling src/core/ext/upb-generated/envoy/type/percent.upb.c
[C]       Compiling src/core/ext/upb-generated/envoy/type/range.upb.c
[CXX]     Compiling src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc
[CXX]     Compiling src/core/ext/filters/client_channel/lb_policy/round_robin/round_robin.cc
In file included from src/core/ext/upb-generated/envoy/type/percent.upb.h:12:0,
                 from src/core/ext/upb-generated/envoy/type/percent.upb.c:11:
third_party/upb/upb/generated_util.h: In function ‘_upb_array_resize_accessor’:
third_party/upb/upb/generated_util.h:44:61: warning: unused parameter ‘type’ [-Wunused-parameter]
                                             upb_fieldtype_t type,
                                                             ^~~~
In file included from src/core/ext/upb-generated/envoy/type/range.upb.h:12:0,
                 from src/core/ext/upb-generated/envoy/type/range.upb.c:11:
third_party/upb/upb/gene

### Launch Milvus server

Add `lib/` directory to `LD_LIBRARY_PATH`

In [1]:
% cd /content/milvus/core/milvus
! echo $LD_LIBRARY_PATH
import os
os.environ['LD_LIBRARY_PATH'] +=":/content/milvus/core/milvus/lib"
! echo $LD_LIBRARY_PATH

[Errno 2] No such file or directory: '/content/milvus/core/milvus'
/content
/usr/local/nvidia/lib:/usr/local/nvidia/lib64
/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/milvus/core/milvus/lib


Start Milvus server and hang

In [2]:
% cd scripts
! ls
! nohup ./start_server.sh &

[Errno 2] No such file or directory: 'scripts'
/content
sample_data
nohup: appending output to 'nohup.out'
nohup: failed to run command './start_server.sh': No such file or directory


Show Milvus server status

If you can find the following content, the Milvus server is successfully running: `Milvus server started successfully!`

In [3]:
! ls
! cat nohup.out

nohup.out  sample_data


## Run Python Code

### Install Milvus Python SDK

In [4]:
! pip install pymilvus

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 2.5MB 5.3MB/s 
     |████████████████████████████████| 184kB 40.9MB/s 


### Import pymilvus

In [5]:
import random
import numpy as np

from milvus import Milvus, IndexType, MetricType, Status

# Milvus server IP address and port.
# You may need to change _HOST and _PORT accordingly.
_HOST = '127.0.0.1'
_PORT = '19530'  # default value
# _PORT = '19121'  # default http value

# Vector parameters
_DIM = 8  # dimension of vector

_INDEX_FILE_SIZE = 32  # max file size of stored index

# the demo name.
collection_name = 'example_collection_'
partition_tag = 'demo_tag_'
segment_name= ''

# 10 vectors with 8 dimension, per element is float32 type, vectors should be a 2-D array
vectors = [[random.random() for _ in range(_DIM)] for _ in range(10)]
ids = [i for i in range(10)]

### Connect Milvus and check version

In [6]:
# Connect to Milvus Server 
milvus = Milvus(_HOST, _PORT)

# Close client instance
# milvus.close()

# Returns the status of the Milvus server.
server_status = milvus.server_status(timeout=10)
print(server_status)

Retry to connect server 127.0.0.1:19530 failed.


NotConnectError: ignored

In [ ]:
# Returns the version of the client.
client_version= milvus.client_version()
print(client_version)

# Returns the version of the Milvus server.
server_version = milvus.server_version(timeout=10)
print(server_version)

### Check and create collection/index

In [ ]:
# Checks whether a collection exists.
milvus.has_collection(collection_name=collection_name, timeout=10)

In [ ]:
# Information needed to create a collection.Defult index_file_size=1024 and metric_type=MetricType.L2
param={'collection_name':collection_name, 'dimension': _DIM, 'index_file_size': _INDEX_FILE_SIZE, 'metric_type': MetricType.L2}
# Creates a collection.
milvus.create_collection(param, timeout=10)

In [ ]:
# Checks whether a partition exists.
milvus.has_partition(collection_name=collection_name, partition_tag=partition_tag)

In [ ]:
# Create a partition for a collection.
milvus.create_partition(collection_name=collection_name, partition_tag=partition_tag, timeout=10)

In [ ]:
ivf_param = {'nlist': 16384}
# Creates index for a collection.
milvus.create_index(collection_name=collection_name, index_type=IndexType.IVF_FLAT, params=ivf_param)

### Insert data and flush

In [ ]:
# Insert vectors to a collection.
milvus.insert(collection_name=collection_name, records=vectors, ids=ids)

In [ ]:
# Flushes vector data in one collection or multiple collections to disk.
milvus.flush(collection_name_array=[collection_name], timeout=None)

### Load data and search

In [ ]:
# Loads a collection for caching.
milvus.load_collection(collection_name=collection_name, timeout=None)

In [ ]:
# Search vectors in a collection.
search_param = { "nprobe": 16 }
milvus.search(collection_name=collection_name,query_records=[vectors[0]],partition_tags=None,top_k=10,params=search_param)

### Get info about collection/index

In [ ]:
# Returns collection list.
milvus.list_collections(timeout=10)

In [ ]:
# Show all partitions in a collection.
milvus.list_partitions(collection_name=collection_name, timeout=10)

In [ ]:
# Returns the number of vectors in a collection.
milvus.count_entities(collection_name=collection_name, timeout=10)

In [ ]:
# Returns information of a collection.
milvus.get_collection_info(collection_name=collection_name, timeout=10)

In [ ]:
# Returns collection statistics information
milvus.get_collection_stats(collection_name=collection_name, timeout=10)

In [ ]:
# Show index information of a collection.
milvus.get_index_info(collection_name=collection_name, timeout=10)

### Get data by ids

In [ ]:
# List the ids in segment
# you can get the segment_name list by get_collection_stats() function.
milvus.list_id_in_segment(collection_name =collection_name, segment_name='1600328539015368000', timeout=None)

In [ ]:
# Returns raw vectors according to ids, and you can get the ids list by list_id_in_segment() function.
# ids (list) -- list of vector id
milvus.get_entity_by_id(collection_name=collection_name, ids=[0], timeout=None)

### Get and set the configurations

In [ ]:
# Gets Milvus configurations.
milvus.get_config(parent_key='cache', child_key='cache_size')

In [ ]:
# Sets Milvus configurations.
milvus.set_config(parent_key='cache', child_key='cache_size', value='5G')

### Delete data/index

In [ ]:
# Removes an index.
milvus.drop_index(collection_name=collection_name, timeout=None)

In [ ]:
# Deletes a partition in a collection.
milvus.drop_partition(collection_name=collection_name, partition_tag=partition_tag, timeout=None)

In [ ]:
# Deletes vectors in a collection by vector ID.
# id_array (list[int]) -- list of vector id
milvus.delete_entity_by_id(collection_name=collection_name, id_array=[0], timeout=None)

In [ ]:
# Deletes a collection by name.
# milvus.drop_collection(collection_name=collection_name, timeout=10)